In [13]:
import sys

sys.path.append("../src/helpers")
import mlflow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mlflow.tracking import MlflowClient
from scipy.stats import pearsonr
from mlflow_utils import mlflow_tracking_uri

client = MlflowClient(tracking_uri=mlflow_tracking_uri)
experiment_id_mapping = experiment_id_mapping = {
    "cat-resnet": "214",
    "rocket-resnet": "216",
    "beaver-resnet": "217",
    "pizza-resnet": "413",
}

def get_results(experiment_name):
    experiment_id = experiment_id_mapping[experiment_name]
    methods = [
        # "original",
        "retrained",
        "finetune",
        "neggrad",
        "relabel",
        "badT",
        "scrub",
        "ssd",
        "unsir",
        "our",
    ]
    # methods.extend(our_method_names)

    metrics = [
        "mia",
        "acc_forget",
        "acc_retain",
        "acc_test",
        "acc_val",
        "js",
        "t",
        "js_proxy",
    ]
    runs = client.search_runs(experiment_id)
    df = pd.DataFrame(
        [
            {k: v for k, v in run.data.metrics.items() if k in metrics}
            for run in runs
            if run.data.params.get("method") in methods
        ]
    )

    df["method"] = [
        run.data.params.get("method")
        for run in runs
        if run.data.params.get("method") in methods
    ]

    df["seed"] = [
        run.data.params.get("seed")
        for run in runs
        if run.data.params.get("method") in methods
    ]

    df.set_index(["method", "seed"], inplace=True)

    grouped_df = df.groupby("method").aggregate(["mean", "std"])
    grouped_df["js"] = grouped_df["js"].apply(lambda x: x * 1e4)
    grouped_df = grouped_df.round(2)

    gap_metrics = ["mia", "acc_forget", "acc_retain", "acc_test", "acc_val"]
    for method in methods:
        for metric in gap_metrics:
            grouped_df.loc[method, f"{metric}_gap"] = abs(
                grouped_df.loc[method, (metric, "mean")]
                - grouped_df.loc["retrained", (metric, "mean")]
            )
    grouped_df["avg_gap"] = (
        grouped_df[
            [
                "mia_gap",
                "acc_retain_gap",
                "acc_forget_gap",
                "acc_test_gap",
            ]
        ]
        .mean(axis=1)
        .round(4)
    )

    main_df = grouped_df.loc[
        [m for m in methods if m in grouped_df.index],
        ["avg_gap", "js", 't', 'js_proxy'],
    # ].sort_values(by=("avg_gap", ""), ascending=True)
    ].sort_values(by=("js", "mean"), ascending=True)
    methods_for_appendix = [m for m in methods if m in grouped_df.index]

    appendix_df = grouped_df.loc[
        methods_for_appendix,
        [
            "avg_gap",
            "mia_gap",
            "acc_forget_gap",
            "acc_retain_gap",
            "acc_test_gap",
            "mia",
            "acc_forget",
            "acc_retain",
            "acc_test",
        ],
    ]
    appendix_df = appendix_df.sort_values(by=("avg_gap", ""), ascending=True)

    print(experiment_name)

    display(main_df)
    display(appendix_df)

    return main_df, appendix_df

# Classes


In [9]:
rn_cat, appendix_df = get_results("cat-resnet")

cat-resnet


avg_gap    js           t       js_proxy      
                   mean   std  mean   std     mean   std
method                                                  
retrained  0.0000  0.00  0.00  4.88  0.13     1.47  0.02
badT       0.1750  2.50  0.44  0.33  0.01     1.03  0.18
unsir      0.0575  2.96  0.16  0.44  0.01     0.57  0.18
our        0.1425  2.99  0.88  0.28  0.00     0.22  0.25
relabel    0.1025  3.78  2.60  0.56  0.00     0.85  0.50
ssd        0.3750  4.31  2.07  1.01  0.01     0.64  0.51
scrub      0.4175  7.44  0.03  0.58  0.01     0.13  0.01
neggrad    0.3400  7.64  0.32  0.48  0.00     0.03  0.03
finetune   0.2550  8.41  0.57  0.42  0.00     0.07  0.02

avg_gap mia_gap acc_forget_gap acc_retain_gap acc_test_gap   mia  \
                                                                      mean   
method                                                                       
retrained  0.0000    0.00           0.00           0.00         0.00  0.35   
unsir      0.0575    0.13           0.06           0.03         0.01  0.48   
relabel    0.1025    0.36           0.03           0.01         0.01  0.71   
our        0.1425    0.35           0.14           0.00         0.08  0.00   
badT       0.1750    0.64           0.00           0.04         0.02  0.99   
finetune   0.2550    0.43           0.54           0.01         0.04  0.78   
neggrad    0.3400    0.34           0.84           0.14         0.04  0.01   
ssd        0.3750    0.15           0.32           0.59         0.44  0.50   
scrub      0.4175    0.56           1.00           0.01         0.10  0.91   

                acc_forget       acc_retain       acc_test        
            std       mean   std       mean   std     mean   std  
method                                                            
retrained  0.02       0.00  0.00       0.99  0.01     0.83  0.01  
unsir      0.14       0.06  0.07       0.96  0.01     0.82  0.01  
relabel    0.51       0.03  0.03       0.98  0.01     0.82  0.02  
our        0.00       0.14  0.11       0.99  0.00     0.91  0.01  
badT       0.01       0.00  0.00       0.95  0.01     0.81  0.01  
finetune   0.19       0.54  0.07       0.98  0.01     0.87  0.01  
neggrad    0.00       0.84  0.10       0.85  0.08     0.79  0.07  
ssd        0.32       0.32  0.56       0.40  0.33     0.39  0.32  
scrub      0.01       1.00  0.00       1.00  0.00     0.93  0.01

In [10]:
rn_beaver = get_results("beaver-resnet")

beaver-resnet


avg_gap      js           t       js_proxy      
                     mean   std  mean   std     mean   std
method                                                    
retrained  0.0000    0.00  0.00  4.00  0.11     0.85  0.24
our        0.1200   25.46  1.41  0.23  0.01     0.51  0.08
ssd        0.4325   45.19  9.19  0.83  0.03     1.41  0.44
scrub      0.3225   64.09  8.71  0.55  0.00     0.18  0.06
unsir      0.4050   76.28  6.88  0.20  0.01     0.36  0.13
badT       0.3000   76.85  3.12  0.26  0.01     0.53  0.15
finetune   0.2850  101.48  2.87  0.43  0.00     0.67  0.17
relabel    0.2925  102.66  3.11  0.45  0.00     0.70  0.20
neggrad    0.3725  108.50  2.69  0.44  0.01     0.88  0.19

avg_gap mia_gap acc_forget_gap acc_retain_gap acc_test_gap   mia  \
                                                                      mean   
method                                                                       
retrained  0.0000    0.00           0.00           0.00         0.00  0.23   
our        0.1200    0.23           0.19           0.02         0.04  0.00   
finetune   0.2850    0.23           0.17           0.50         0.24  0.00   
relabel    0.2925    0.23           0.20           0.50         0.24  0.00   
badT       0.3000    0.23           0.18           0.53         0.26  0.00   
scrub      0.3225    0.39           0.87           0.02         0.01  0.62   
neggrad    0.3725    0.34           0.41           0.50         0.24  0.57   
unsir      0.4050    0.51           0.97           0.08         0.06  0.74   
ssd        0.4325    0.25           0.00           0.90         0.58  0.48   

                acc_forget       acc_retain       acc_test        
            std       mean   std       mean   std     mean   std  
method                                                            
retrained  0.02       0.00  0.00       0.91  0.03     0.59  0.02  
our        0.00       0.19  0.07       0.93  0.03     0.63  0.02  
finetune   0.00       0.17  0.07       0.41  0.07     0.35  0.05  
relabel    0.00       0.20  0.08       0.41  0.06     0.35  0.05  
badT       0.00       0.18  0.10       0.38  0.04     0.33  0.03  
scrub      0.12       0.87  0.08       0.93  0.03     0.60  0.02  
neggrad    0.03       0.41  0.12       0.41  0.06     0.35  0.05  
unsir      0.09       0.97  0.01       0.99  0.01     0.65  0.01  
ssd        0.26       0.00  0.00       0.01  0.00     0.01  0.00

In [12]:
rocket_resnet, appendix_df = get_results("rocket-resnet")

rocket-resnet


avg_gap     js            t       js_proxy      
                    mean    std  mean   std     mean   std
method                                                    
retrained  0.0000   0.00   0.00  5.05  2.01     1.49  0.35
our        0.1475  20.77   6.25  0.22  0.00     0.87  0.11
ssd        0.4700  36.44  11.02  0.81  0.01     1.67  0.76
scrub      0.4000  66.49  14.36  0.55  0.00     0.10  0.03
badT       0.3450  67.73  10.80  0.26  0.00     0.34  0.06
unsir      0.4350  74.53  12.03  0.20  0.01     0.17  0.04
finetune   0.3525  95.27   8.23  0.43  0.00     0.61  0.25
relabel    0.3525  96.02   7.40  0.45  0.00     0.60  0.20
neggrad    0.4450  98.06   8.04  0.44  0.01     0.54  0.24

avg_gap mia_gap acc_forget_gap acc_retain_gap acc_test_gap   mia  \
                                                                      mean   
method                                                                       
retrained  0.0000    0.00           0.00           0.00         0.00  0.18   
our        0.1475    0.18           0.39           0.02         0.00  0.00   
badT       0.3450    0.18           0.40           0.54         0.26  0.00   
finetune   0.3525    0.18           0.43           0.53         0.27  0.00   
relabel    0.3525    0.18           0.43           0.53         0.27  0.00   
scrub      0.4000    0.64           0.93           0.01         0.02  0.82   
unsir      0.4350    0.67           0.99           0.05         0.03  0.85   
neggrad    0.4450    0.44           0.52           0.54         0.28  0.62   
ssd        0.4700    0.51           0.00           0.84         0.53  0.69   

                acc_forget       acc_retain       acc_test        
            std       mean   std       mean   std     mean   std  
method                                                            
retrained  0.10       0.00  0.00       0.94  0.05     0.62  0.05  
our        0.00       0.39  0.27       0.92  0.03     0.62  0.01  
badT       0.00       0.40  0.04       0.40  0.05     0.36  0.05  
finetune   0.00       0.43  0.06       0.41  0.07     0.35  0.05  
relabel    0.00       0.43  0.02       0.41  0.07     0.35  0.05  
scrub      0.09       0.93  0.08       0.93  0.03     0.60  0.02  
unsir      0.03       0.99  0.01       0.99  0.01     0.65  0.01  
neggrad    0.01       0.52  0.10       0.40  0.07     0.34  0.05  
ssd        0.47       0.00  0.00       0.10  0.08     0.09  0.07

In [14]:
pizza_resnet, appendix_df = get_results("pizza-resnet")

pizza-resnet


avg_gap      js             t        js_proxy      
                     mean    std   mean    std     mean   std
method                                                       
retrained  0.0000    0.00   0.00  42.15  16.05     1.71  0.30
ssd        0.3650   34.96  14.21   3.19   0.03     1.57  0.58
our        0.0925   37.02  18.68   1.30   0.02     0.66  0.56
badT       0.3125   72.62  22.07   1.59   0.01     0.48  0.25
scrub      0.4200   73.10   0.82   4.05   0.03     0.15  0.10
neggrad    0.3250   86.36   9.66   3.24   0.03     0.27  0.14
relabel    0.2925   92.27   6.43   3.27   0.03     0.28  0.14
finetune   0.2975   94.96   7.24   3.23   0.01     0.31  0.17
unsir      0.5075  102.29   9.33   1.01   0.01     0.28  0.14

avg_gap mia_gap acc_forget_gap acc_retain_gap acc_test_gap   mia  \
                                                                      mean   
method                                                                       
retrained  0.0000    0.00           0.00           0.00         0.00  0.22   
our        0.0925    0.04           0.19           0.09         0.05  0.26   
relabel    0.2925    0.11           0.67           0.22         0.17  0.33   
finetune   0.2975    0.11           0.69           0.22         0.17  0.33   
badT       0.3125    0.39           0.29           0.37         0.20  0.61   
neggrad    0.3250    0.26           0.64           0.23         0.17  0.48   
ssd        0.3650    0.46           0.00           0.63         0.37  0.68   
scrub      0.4200    0.71           0.96           0.01         0.00  0.93   
unsir      0.5075    0.78           0.80           0.27         0.18  1.00   

                acc_forget       acc_retain       acc_test        
            std       mean   std       mean   std     mean   std  
method                                                            
retrained  0.16       0.00  0.00       0.92  0.14     0.57  0.07  
our        0.46       0.19  0.28       0.83  0.11     0.52  0.02  
relabel    0.58       0.67  0.23       0.70  0.05     0.40  0.01  
finetune   0.58       0.69  0.22       0.70  0.05     0.40  0.01  
badT       0.54       0.29  0.28       0.55  0.06     0.37  0.02  
neggrad    0.50       0.64  0.11       0.69  0.05     0.40  0.01  
ssd        0.51       0.00  0.00       0.29  0.31     0.20  0.22  
scrub      0.08       0.96  0.07       0.91  0.15     0.57  0.07  
unsir      0.00       0.80  0.17       0.65  0.04     0.39  0.02